In [ ]:
import tensorflow as tf 
import numpy as np

#Reproducibility 

tf.random.set_seed(777)
# Define characters from the quote without duplicates
idx2char = sorted(set(
    "If you want to build a ship, don’t drum up people to collect wood and don’t assign them tasks and work, "
    "but rather teach them to long for the endless immensity of the sea."
))

# Create a mapping from character to index
char2idx = {ch: idx for idx, ch in enumerate(idx2char)}

# Quote
quote = "If you want to build a ship, don’t drum up people to collect wood and don’t assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

# Prepare x_data and y_data based on the quote
x_data = []
y_data = []

# Create training data by sliding window
sequence_length = len(quote)-1  # Length of the input sequence
for i in range(len(quote) - sequence_length):
    x_seq = quote[i:i + sequence_length]
    y_seq = quote[i + 1:i + sequence_length + 1]
    
    x_data.append([char2idx[ch] for ch in x_seq])  # Convert characters to indices
    y_data.append([char2idx[ch] for ch in y_seq])  # Shifted sequence for targets


In [ ]:



# Convert to numpy arrays
x_data = np.array(x_data)
y_data = np.array(y_data)

num_classes = len(idx2char)  # Output unique chars based on idx2char
input_dim = num_classes  # One-hot size
hidden_size = 128  # LSTM output size
batch_size = 1   # One sentence
learning_rate = 0.1

# Use tf.one_hot to create one-hot encoded input
x_one_hot = tf.one_hot(x_data, depth=num_classes)


In [ ]:
#Build the LSTM Model
class LSTMModel(tf.keras.Model):
    def __init__(self):
        super(LSTMModel,self).__init__()
        self.lstm=tf.keras.layers.LSTM(units=hidden_size,return_sequences=True)
        self.fc =tf.keras.layers.Dense(units=num_classes)
        
    def call(self,x):
        x=self.lstm(x)
        x=self.fc(x)
        x=tf.nn.softmax(x)
        return x
    

In [ ]:
# Prepare data
X = np.array(x_one_hot, dtype=np.float32)
Y = np.array(y_data, dtype=np.int32)

In [ ]:

# Create model
model = LSTMModel()

# Loss and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


In [ ]:
# Training step
def train_step(model, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, predictions

# Training loop
for epoch in range(100):
    loss, predictions = train_step(model, X, Y)
    predicted = tf.argmax(predictions, axis=2).numpy()

    print(f"Epoch {epoch}, Loss: {loss.numpy()}")
    print(f"Prediction: {predicted}, True Y: {y_data}")

    # Convert prediction to characters
    result_str = [idx2char[c] for c in predicted[0]]
    print("\tPrediction str: ", ''.join(result_str))    